# Router Agent Benchmark
This notebook implements a benchmark using groundtruthing technique to evaluate AI models that function as **Router Agents**

## Setup LLM/SLM Provider
In this example I'll use LangChain and OpenAI as LLM provider, switch to your provider (AWS Bedrock with boto3, Groq directly calls, etc.)

To guarantee only the three words as result, I'll define an enum and use it as output parser (If the model answers something different from it, it will retry automatically.)

In [6]:
from enum import Enum
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

class CategoryEnum(str, Enum):
    MATH = "math"
    HISTORY = "history"
    JOKE = "joke"

class CategoryModel(BaseModel):
    category_result: CategoryEnum = Field(description="The agent's category result")

#TODO: should bea function and receive the model as parameter
parser = PydanticOutputParser(pydantic_object=CategoryModel)

prompt = ChatPromptTemplate.from_messages([
    ("system", """
        You are a Router Agent specialized in classifying user questions into specific categories.
        Your function is to analyze the user's question and determine which category best applies:
            - **math**: Questions related to mathematical calculations, arithmetic operations, algebra, geometry, statistics, or any problem that requires mathematical reasoning or numerical computation.
            - **history**: Questions about historical events, historical figures, important dates, historical periods, wars, revolutions, or any topic related to the past and history.
            - **joke**: Requests to tell jokes, make humor, or any request related to humorous entertainment, regardless of language.
        
        **Important instructions:**
            1. Carefully analyze the intention and content of the question
            2. Identify the most appropriate category based on the main purpose of the question
            3. Return EXCLUSIVELY the category in the specified format, without additional text or explanations
            4. Be accurate and consistent in classification

        Expected output format:
        {format_instructions}
    """),
    ("human", """
        Classify the following question into one of the available categories (math, history, or joke):
        Question: {input}""")
    ])

model = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
)

chain = prompt | model | parser

## Prepare Groundtruth Data
Loads groundtruth data from JSON file to memory

In [7]:
import json 

groundtruth_file_path = "groundtruth.json"

with open(groundtruth_file_path, "r", encoding="utf-8") as file:
    groundtruth_data = json.load(file)

## Define Models to Test

Define a list of models to test (should follow the provider names definition, in my case, OpenAI.)

In [8]:
models_to_test = [
    "gpt-4o",
    "gpt-4o-mini",
    "gpt-5.2",
    "gpt-5-mini",
    "gpt-5-nano"
]

models_to_test = ["gpt-4o-mini"]

## Call Models and Collect Metrics

In [ ]:
from tqdm import tqdm 

class ModelResult(BaseModel):
    model_name: str
    precison: float
    latency_p50: float
    latency_p90: float
    latency_p95: float


for model in models_to_test:
    print(f"Testing {model} model")
    inputs = []
    expected_outputs = []
    model_output = []
    is_matches = []

    
    for case in tqdm(groundtruth_data):
        inputs.append(case["input"])
        expected_outputs.append(case["expected_output"])

        #TODO: Should change to a function to invoke model by parameter
        result = chain.invoke({
            "input": case["input"],
            "format_instructions": parser.get_format_instructions()
        })

        if result.category_result == case["expected_output"]:
            is_matches.append(True)
        else:
            is_matches.append(False)


Testing gpt-4o-mini model


 64%|██████▍   | 44/69 [00:31<00:16,  1.56it/s]